# 複素ANNによる非線形歪補償
複素数を入力とする3層ANNによる補償

In [1]:
#import
import sys
import os
import time
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

sys.path.append('../')
from pyopt.util import save_pickle, load_pickle

# 1. Preprocessing

## 1.1 データの整形

In [19]:
def data_shaping(input_signal, signal, max_tap, tap):
    """
    input_signal: 伝送前の信号
    signal: 伝送後の信号
    max_tap: 最大の同時入力シンボル数
    tap: 同時入力シンボル数
    
    signal = [x_0, x_1, ... , x_(n-1)]
      |
      |
      v
    x = [[x_0, x_1, ... , x_tap-1],
            [x_1, x_2, ..., x_tap],
                   .
                   .
                   .
            [x_(n-tap), x_(n-tap+1), ..., x(n-1)]]
      |
      |
      v
    x = [[i_0, q_0, i_1, q_1, ... , i_(tap-1), q_(tap-1)],
            [i_1, q_1, i_2, q_2, ... , i_tap, q_tap],
                   .
                   .
                   .
            [i_(n-tap), q_(n-tap), i_(n-tap+1), q_(n-tap+1), ..., i_(n-1), q_(n-1)]] (batch, input_dim) input_dim = tap * 2
    
    y  (batch, output_dim) output_dim = 2
    """
    
    x = np.zeros((len(input_signal) - (max_tap - 1), tap, 2), dtype=float)
    y = np.zeros((len(input_signal) - (max_tap - 1), 2), dtype=float)
    for i, j in enumerate(np.arange(max_tap // 2, len(input_signal) - max_tap // 2)):
        x[i, :, 0] = signal[j - tap // 2: j + tap // 2 + 1].real
        x[i, :, 1] = signal[j - tap // 2: j + tap // 2 + 1].imag
        y[i, 0] = input_signal[j].real
        y[i, 1] = input_signal[j].imag
    return x, y

In [94]:
#動作確認
tap = 29
max_tap = 51

df_dir = '../data/input/prbs.csv'
df = pd.read_csv(df_dir, index_col=0)  # dataframe読み込み
condition = (df['N']==13) & (df['itr']==1) & (df['form']=='RZ16QAM') & (df['n']==32) & (df['equalize']==False) & (df['baudrate']==28) & (df['PdBm']==1)
sgnl = load_pickle(df[condition].iloc[0]['data_path'])  # dataframeから条件と合う行を取得し,pickleの保存先(data_path)にアクセス
lc = sgnl.linear_compensation(500, sgnl.signal['x_500'])
x, y = data_shaping(sgnl.signal['x_0'][16::32], lc[16::32], max_tap, tap)  # ANNに入力できるようにデータを整形

print('x size: ', x.shape)
print('y size: ', y.shape)
print(x[0])
print(y[0])

x size:  (1998, 29, 2)
y size:  (1998, 2)
[[ 51598.75641186 -53403.3657517 ]
 [-32145.84759244  96508.43591778]
 [-30606.37446336  95283.58932182]
 [ 32503.60952844  13910.52002391]
 [  8248.62335619  73391.81300168]
 [ 29107.6877347   14808.73597731]
 [-78280.98856905  15366.79259349]
 [-76605.44555413  12791.23534176]
 [ 69345.95709822  -8729.55171237]
 [ 36932.5922594   11156.38959978]
 [ 13056.01107017 -32846.73484666]
 [ 10610.89876389 -34963.38298674]
 [-73494.72750711   7190.0287674 ]
 [-49860.52436413  54128.52098679]
 [-53019.95429361  53670.26520552]
 [ 32274.63575034 -91081.39804461]
 [-14385.94066463  30341.90659466]
 [-47630.74129796  49235.95785073]
 [-11546.66807361 -75020.35362366]
 [-11274.86250656  31430.37354787]
 [-51761.20934421  52640.09062926]
 [-92079.7374958  -28400.06490156]
 [-54174.37271402  50576.41182636]
 [-67054.54768817  15313.41343857]
 [ 96034.7954907   33242.87671139]
 [ 50154.90489795 -49671.1420593 ]
 [-93925.54099544 -34256.15458994]
 [-30446.5662

In [29]:
x_tensor = torch.Tensor(x)
x_complex = torch.view_as_complex(x_tensor)
print(x_complex)

tensor([[ 51598.7578-53403.3672j, -32145.8477+96508.4375j,
         -30606.3750+95283.5859j,  ...,
         -93925.5391-34256.1562j, -30446.5664-13048.7129j,
           9693.4248-29001.5801j],
        [-32145.8477+96508.4375j, -30606.3750+95283.5859j,
          32503.6094+13910.5205j,  ...,
         -30446.5664-13048.7129j,   9693.4248-29001.5801j,
          53869.6953+53203.7969j],
        [-30606.3750+95283.5859j,  32503.6094+13910.5205j,
           8248.6230+73391.8125j,  ...,
           9693.4248-29001.5801j,  53869.6953+53203.7969j,
         -74834.9922+12591.7324j],
        ...,
        [-57464.2109-54496.4727j, -71785.1406+16135.0635j,
         -37748.6445-11243.9277j,  ...,
         -10821.5039-70501.7188j, -72369.8047+16814.8672j,
         -10232.1523+33094.5391j],
        [-71785.1406+16135.0635j, -37748.6445-11243.9277j,
          96017.2891+27587.5176j,  ...,
         -72369.8047+16814.8672j, -10232.1523+33094.5391j,
         -12742.2695-76211.6172j],
        [-37748.6445-1

## 1.2 平均,標準偏差の計算

In [25]:
mean = np.mean(x)
std = np.std(x)

print('mean: ', mean)
print('std: ', std)

mean:  1153.2726078905946
std:  52094.955682540436


# 2. Dataset定義

In [95]:
class Dataset(data.Dataset):
    def __init__(self, x, y, mean, std):
        self.x, self.y, self.mean, self.std = x, y, mean, std
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std
        
        x = torch.Tensor(x)
        y = torch.Tensor(y)
        x = torch.view_as_complex(x)
        y = torch.view_as_complex(y)
        return x, y

In [96]:
#動作確認
train_dataset = Dataset(x=x, y=y, mean=mean, std=std)

index = 0
x_normalized, y_normalized = train_dataset.__getitem__(index)
x_array = x_normalized.detach().numpy()

print('mean: ', np.mean(x_array))
print('std: ', np.std(x_array))
print(x_normalized)
print(y_normalized)

mean:  (-0.3059248+0.15033558j)
std:  1.3204029
tensor([ 0.9683-1.0473j, -0.6392+1.8304j, -0.6096+1.8069j,  0.6018+0.2449j,
         0.1362+1.3867j,  0.5366+0.2621j, -1.5248+0.2728j, -1.4926+0.2234j,
         1.3090-0.1897j,  0.6868+0.1920j,  0.2285-0.6527j,  0.1815-0.6933j,
        -1.4329+0.1159j, -0.9792+1.0169j, -1.0399+1.0081j,  0.5974-1.7705j,
        -0.2983+0.5603j, -0.9364+0.9230j, -0.2438-1.4622j, -0.2386+0.5812j,
        -1.0157+0.9883j, -1.7897-0.5673j, -1.0621+0.9487j, -1.3093+0.2718j,
         1.8213+0.6160j,  0.9406-0.9756j, -1.8251-0.6797j, -0.6066-0.2726j,
         0.1639-0.5788j])
tensor(-1.3750+0.4288j)


In [97]:
batch_size = 100

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader}

# 3. Model定義

In [98]:
def zrelu(x):
    in_shape = x.shape
    x = torch.view_as_real(x)
    x = x.view(-1, 2)
    for i in range(x.shape[0]):
        if x[i, 0] < 0 or x[i, 1] < 0:
            x[i, 0] = 0
            x[i, 1] = 0
    x = x.view(in_shape[0], in_shape[1], -1)
    return torch.view_as_complex(x)

In [99]:
def crelu(x):
    in_shape = x.shape
    x = torch.view_as_real(x)
    x = x.view(-1, 2)
    x = F.relu(x)
    x = x.view(in_shape[0], in_shape[1], -1)
    print(x)
    return torch.view_as_complex(x)

In [100]:
class ANN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_neuron):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_neuron)
        self.fc2 = nn.Linear(hidden_neuron, output_dim)
    
    def forward(self, x):
        x = zrelu(self.fc1(x))
        x = self.fc2(x)
        return x

In [93]:
x = torch.Tensor(x)
x_comp = torch.view_as_complex(x)
x_relu = zrelu(x_comp)
print(x_relu[0])

tensor([    0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
        32503.6094+13910.5205j,  8248.6230+73391.8125j, 29107.6875+14808.7363j,
            0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
        36932.5938+11156.3896j,     0.0000+0.0000j,     0.0000+0.0000j,
            0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
            0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
            0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
            0.0000+0.0000j,     0.0000+0.0000j,     0.0000+0.0000j,
        96034.7969+33242.8750j,     0.0000+0.0000j,     0.0000+0.0000j,
            0.0000+0.0000j,     0.0000+0.0000j])


In [88]:
x = torch.Tensor(x)
x_comp = torch.view_as_complex(x)
x_real = torch.view_as_real(x_comp)
print(x_real.shape)
x_view = x_real.view(-1, 2)
print(x_view.shape)
print(x_view[0])
print(F.relu(x_view[0]))

torch.Size([1998, 29, 2])
torch.Size([57942, 2])
tensor([ 51598.7578, -53403.3672])
tensor([51598.7578,     0.0000])


In [62]:
x_view[0, 0] = 0
x_view[0, 1] = 0
print(x_view[0])

tensor([0., 0.])


In [101]:
#動作確認
hidden_neuron = 300
device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

model = ANN(input_dim=tap*2, output_dim=2, hidden_neuron=hidden_neuron).to(device)
for x, y in train_dataloader:
    output = model(x)
    print(output[:6])
    break

Device available now: cpu


RuntimeError: Expected object of scalar type Float but got scalar type ComplexFloat for argument #2 'mat1' in call to _th_addmm